In [1]:
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from rigl_torch.models import ModelFactory
from rigl_torch.optim.cosine_annealing_with_linear_warm_up import CosineAnnealingWithLinearWarmUp
from rigl_torch.rigl_constant_fan import RigLConstFanScheduler
from rigl_torch.rigl_scheduler import RigLScheduler

INFO:/home/condensed-sparsity/src/rigl_torch/models/model_factory.py:Registering mnist for mnist dataset to ModelFactory...
INFO:/home/condensed-sparsity/src/rigl_torch/models/model_factory.py:Registering resnet18 for cifar10 dataset to ModelFactory...
INFO:/home/condensed-sparsity/src/rigl_torch/models/model_factory.py:Registering wide_resnet22 for cifar10 dataset to ModelFactory...
/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
INFO:/home/condensed-sparsity/src/rigl_torch/models/model_factory.py:Registering cond_net for mnist dataset to ModelFactory...


In [2]:
from rigl_torch.datasets import get_dataloaders
from omegaconf import DictConfig
import hydra

with hydra.initialize(config_path="../configs"):
    cfg = hydra.compose(config_name="config.yaml", overrides=[])
cfg

/tmp/ipykernel_122759/856747501.py:5: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize(config_path="../configs"):


{'dataset': {'name': 'cifar10', 'normalize': False, 'num_classes': 10, 'classes': ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], 'train_len': 50000}, 'model': {'name': 'resnet18'}, 'experiment': {'comment': 'missing_sweep_${rigl.dense_allocation}_${rigl.const_fan_in}', 'name': '${model.name}_${dataset.name}_${experiment.comment}', 'sweep': True}, 'paths': {'data_folder': '/home/condensed-sparsity/data', 'artifacts': '/home/condensed-sparsity/artifacts', 'logs': '/home/condensed-sparsity/logs', 'checkpoints': '${paths.artifacts}/checkpoints'}, 'rigl': {'dense_allocation': 0.5, 'delta': 100, 'grad_accumulation_n': 1, 'alpha': 0.3, 'static_topo': 0, 'const_fan_in': False, 'sparsity_distribution': 'erk', 'erk_power_scale': 1.0}, 'training': {'dry_run': False, 'batch_size': 128, 'test_batch_size': 1000, 'epochs': 250, 'seed': 42, 'log_interval': 10000, 'save_model': True, 'optimizer': 'sgd', 'weight_decay': 0.0005, 'momentum': 0.9, 'scheduler': '

In [3]:
cfg.rigl.sparsity_distribution = "erk"

In [4]:
# model = ModelFactory.load_model(model="mnist", dataset='mnist').to(device)
use_cuda = not cfg.compute.no_cuda and torch.cuda.is_available()
torch.manual_seed(cfg.training.seed)
device = torch.device("cuda" if use_cuda else "cpu")
train_loader, test_loader = get_dataloaders(cfg)

model = ModelFactory.load_model(model=cfg.model.name, dataset=cfg.dataset.name).to(device)
# model = get_model(cfg).to(device)
optimizer = torch.optim.Adadelta(model.parameters(), lr=cfg.training.lr)
scheduler = CosineAnnealingWithLinearWarmUp(
    optimizer,
    T_max=cfg.training.epochs,
    eta_min=0,
    lr=cfg.training.lr,
    warm_up_steps=cfg.training.warm_up_steps,
)

pruner = lambda: True  # noqa: E731
if cfg.rigl.dense_allocation is not None:
    T_end = int(0.75 * cfg.training.epochs * len(train_loader))
    if cfg.rigl.const_fan_in:
        rigl_scheduler = RigLConstFanScheduler
    else:
        rigl_scheduler = RigLScheduler
    pruner = rigl_scheduler(
        model,
        optimizer,
        dense_allocation=cfg.rigl.dense_allocation,
        alpha=cfg.rigl.alpha,
        delta=cfg.rigl.delta,
        static_topo=cfg.rigl.static_topo,
        T_end=T_end,
        ignore_linear_layers=False,
        grad_accumulation_n=cfg.rigl.grad_accumulation_n,
        sparsity_distribution=cfg.rigl.sparsity_distribution,
        erk_power_scale=cfg.rigl.erk_power_scale,
    )
else:
    print(
        "cfg.rigl.dense_allocation is `null`, training with dense "
        "network..."
    )

Files already downloaded and verified


INFO:/home/condensed-sparsity/src/rigl_torch/models/model_factory.py:Loading model resnet18/cifar10 using <function ResNet18 at 0x7f00e14fb160> with args: () and kwargs: {}
INFO:/home/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 20 set to 0.0
INFO:/home/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 0 set to 0.0
INFO:/home/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 7 set to 0.0
INFO:/home/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 12 set to 0.0
INFO:/home/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 17 set to 0.0
INFO:/home/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 1 set to 0.0
INFO:/home/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 2 set to 0.0
INFO:/home/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 3 set to 0.0


In [5]:
from rigl_torch.utils.checkpoint import Checkpoint
checkpoint = Checkpoint(
    run_id = "test",
    cfg = cfg,
    model = model,
    optimizer=optimizer,
    scheduler=scheduler,
    pruner=pruner,
)

In [6]:
checkpoint.current_acc

0.0

In [7]:
checkpoint.current_acc = 1.0

In [8]:
checkpoint._update_best_flag()

INFO:rigl_torch.utils.checkpoint:New best checkpoint accuracy (1.000000 > -inf)!


In [9]:
checkpoint.is_best

True

In [10]:
checkpoint.save_checkpoint()

INFO:rigl_torch.utils.checkpoint:Checkpoint state saved!


In [11]:
Checkpoint.parent_dir.__str__()

'/home/condensed-sparsity/artifacts/checkpoints'

In [12]:
checkpoint = Checkpoint.load_last_checkpoint(run_id="test")

INFO:rigl_torch.utils.checkpoint:Loading checkpoint from /home/condensed-sparsity/artifacts/checkpoints/20220819_test/checkpoint.pt.tar...


In [14]:
checkpoint._update_state()

{'run_id': 'test',
 'cfg': {'dataset': {'name': 'cifar10', 'normalize': False, 'num_classes': 10, 'classes': ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], 'train_len': 50000}, 'model': {'name': 'resnet18'}, 'experiment': {'comment': 'missing_sweep_${rigl.dense_allocation}_${rigl.const_fan_in}', 'name': '${model.name}_${dataset.name}_${experiment.comment}', 'sweep': True}, 'paths': {'data_folder': '/home/condensed-sparsity/data', 'artifacts': '/home/condensed-sparsity/artifacts', 'logs': '/home/condensed-sparsity/logs', 'checkpoints': '${paths.artifacts}/checkpoints'}, 'rigl': {'dense_allocation': 0.5, 'delta': 100, 'grad_accumulation_n': 1, 'alpha': 0.3, 'static_topo': 0, 'const_fan_in': False, 'sparsity_distribution': 'erk', 'erk_power_scale': 1.0}, 'training': {'dry_run': False, 'batch_size': 128, 'test_batch_size': 1000, 'epochs': 250, 'seed': 42, 'log_interval': 10000, 'save_model': True, 'optimizer': 'sgd', 'weight_decay': 0.0005, 'mom

In [5]:

def train(
    cfg, model, device, train_loader, optimizer, epoch, pruner, scheduler, step
):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        step += 1
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        logits = model(data)
        output = F.log_softmax(logits, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()

        if pruner():
            optimizer.step()
        scheduler.step()

        if step % cfg.training.log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )
        if cfg.training.dry_run:
            print("Dry run, exiting after one training step")
            return step
    return step

In [28]:
cfg.training.epochs = 2 
for epoch in range(1, cfg.training.epochs + 1):
        print(pruner)
        step = train(
            cfg,
            model,
            device,
            train_loader,
            optimizer,
            epoch,
            pruner=pruner,
            scheduler=scheduler,
            step=step,
        )
checkpoint = Checkpoint()

RigLScheduler(
layers=21,
nonzero_params=[1728/1728, 36864/36864, 36864/36864, 36864/36864, 36864/36864, 73728/73728, 147456/147456, 8192/8192, 147456/147456, 147456/147456, 294912/294912, 424959/589824, 32768/32768, 424959/589824, 424959/589824, 634976/1179648, 844994/2359296, 131072/131072, 844994/2359296, 844994/2359296, 5120/5120],
nonzero_percentages=[100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 72.05%, 100.00%, 72.05%, 72.05%, 53.83%, 35.82%, 100.00%, 35.82%, 35.82%, 100.00%],
total_nonzero_params=5582179/11164352 (50.00%),
total_CONV_nonzero_params=5577059/11159232 (49.98%),
step=782,
num_rigl_steps=7,
ignoring_linear_layers=False,
sparsity_distribution=erk,
)
RigLScheduler(
layers=21,
nonzero_params=[1728/1728, 36864/36864, 36864/36864, 36864/36864, 36864/36864, 73728/73728, 147456/147456, 8192/8192, 147456/147456, 147456/147456, 294912/294912, 424959/589824, 32768/32768, 424959/589824, 424959/589824, 634976/1179648, 844994/

In [43]:
d = None
d.__dict__()

AttributeError: 'NoneType' object has no attribute '__dict__'

In [62]:
import pathlib

checkpoint_dir = pathlib.Path("./")

In [65]:
test="str"
type(test)

str

In [63]:
type(checkpoint_dir)

pathlib.PosixPath

In [61]:
import datetime

datetime.date.today().isoformat()
# datetime.datetime.now()

'2022-08-19'

In [42]:
pruner.__dir__()

['_implemented_sparsity_distributions',
 '_logger',
 'erk_power_scale',
 'model',
 'optimizer',
 'W',
 '_linear_layers_mask',
 'dense_allocation',
 'N',
 'sparsity_distribution',
 'static_topo',
 'grad_accumulation_n',
 'ignore_linear_layers',
 'backward_masks',
 'S',
 'step',
 'rigl_steps',
 'delta_T',
 'alpha',
 'T_end',
 'backward_hook_objects',
 '__module__',
 '__init__',
 '_allocate_sparsity',
 '_uniform_sparsity_dist',
 '_erk_sparsity_dist',
 'state_dict',
 'load_state_dict',
 'random_sparsify',
 '__str__',
 'reset_momentum',
 'apply_mask_to_weights',
 'apply_mask_to_gradients',
 'check_if_backward_hook_should_accumulate_grad',
 'cosine_annealing',
 '__call__',
 '_rigl_step',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__repr__',
 '__hash__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__d

In [39]:
getattr(pruner, "state_dict2")

AttributeError: 'RigLScheduler' object has no attribute 'state_dict2'

In [20]:
pruner.state_dict()["step"]

782

In [17]:
pruner.state_dict()["rigl_steps"]

7

In [16]:
pruner.state_dict().keys()

dict_keys(['dense_allocation', 'S', 'N', 'hyperparams', 'step', 'rigl_steps', 'backward_masks', '_linear_layers_mask'])

In [46]:
print(pruner.state_dict()["hyperparams"])

{'delta_T': 100, 'alpha': 0.3, 'T_end': 73312, 'ignore_linear_layers': False, 'static_topo': 0, 'sparsity_distribution': 'erk', 'grad_accumulation_n': 1}


In [6]:
print(pruner)

RigLScheduler(
layers=21,
nonzero_params=[1728/1728, 36864/36864, 36864/36864, 36864/36864, 36864/36864, 73728/73728, 147456/147456, 8192/8192, 147456/147456, 147456/147456, 294912/294912, 424959/589824, 32768/32768, 424959/589824, 424959/589824, 634976/1179648, 844994/2359296, 131072/131072, 844994/2359296, 844994/2359296, 5120/5120],
nonzero_percentages=[100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 100.00%, 72.05%, 100.00%, 72.05%, 72.05%, 53.83%, 35.82%, 100.00%, 35.82%, 35.82%, 100.00%],
total_nonzero_params=5582179/11164352 (50.00%),
total_CONV_nonzero_params=5577059/11159232 (49.98%),
step=0,
num_rigl_steps=0,
ignoring_linear_layers=False,
sparsity_distribution=erk,
)


In [7]:
from rigl_torch.util import get_W
W = get_W(model, return_linear_layers_mask=False)
for w in W:
    print(w.shape)
    print(w.numel())

torch.Size([64, 3, 3, 3])
1728
torch.Size([64, 64, 3, 3])
36864
torch.Size([64, 64, 3, 3])
36864
torch.Size([64, 64, 3, 3])
36864
torch.Size([64, 64, 3, 3])
36864
torch.Size([128, 64, 3, 3])
73728
torch.Size([128, 128, 3, 3])
147456
torch.Size([128, 64, 1, 1])
8192
torch.Size([128, 128, 3, 3])
147456
torch.Size([128, 128, 3, 3])
147456
torch.Size([256, 128, 3, 3])
294912
torch.Size([256, 256, 3, 3])
589824
torch.Size([256, 128, 1, 1])
32768
torch.Size([256, 256, 3, 3])
589824
torch.Size([256, 256, 3, 3])
589824
torch.Size([512, 256, 3, 3])
1179648
torch.Size([512, 512, 3, 3])
2359296
torch.Size([512, 256, 1, 1])
131072
torch.Size([512, 512, 3, 3])
2359296
torch.Size([512, 512, 3, 3])
2359296
torch.Size([10, 512])
5120


In [8]:
total_el = 0
non_zero_el = 0
for mask, weights in list(zip(pruner.backward_masks, pruner.W)):
    if mask is None:
        total_el += weights.numel()
        non_zero_el += weights.numel()
    else:
        total_el +=weights.numel()
        non_zero_el+=mask.sum()

In [9]:
total_el

11164352

In [10]:
non_zero_el

tensor(5582179, device='cuda:0')

In [11]:
non_zero_el / total_el

tensor(0.5000, device='cuda:0')

In [19]:
pruner.state_dict().keys()

dict_keys(['dense_allocation', 'S', 'N', 'hyperparams', 'step', 'rigl_steps', 'backward_masks', '_linear_layers_mask'])

In [32]:
state_dict = pruner.state_dict()
state_dict["_linear_layers_mask"]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [16]:
pruner.backward_masks

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 tensor([[[[ True, False,  True],
           [ True,  True, False],
           [False, False,  True]],
 
          [[ True, False, False],
           [ True,  True,  True],
           [False,  True,  True]],
 
          [[False, False,  True],
           [False,  True, False],
           [ True, False,  True]],
 
          ...,
 
          [[ True,  True,  True],
           [ True,  True,  True],
           [ True, False,  True]],
 
          [[False,  True,  True],
           [False,  True,  True],
           [False,  True, False]],
 
          [[ True,  True,  True],
           [ True,  True, False],
           [False,  True,  True]]],
 
 
         [[[ True, False, False],
           [ True,  True,  True],
           [ True,  True,  True]],
 
          [[False,  True, False],
           [ True,  True,  True],
           [False, False,  True]],
 
          [[ True,  True,  True],
           [ True, False,  Tr